In [1]:
import torch
from supervoice_valle import SupervoceNARModel, Tokenizer
from train.dataset import load_sampler
from IPython.display import Audio, display
import matplotlib.pyplot as plt
from vocos import Vocos

In [2]:
vocos = Vocos.from_pretrained("charactr/vocos-encodec-24khz")
tokenizer = Tokenizer("./tokenizer_text.model")
sampler = load_sampler("./external_datasets/libriheavy/libriheavy_cuts_small.jsonl.gz", "./external_datasets/libriheavy-encodec/", 1, tokenizer)
model = SupervoceNARModel()
checkpoint = torch.load("./output/valle-18.pt", map_location = "cpu")
model.load_state_dict(checkpoint['model'])
# model = model.to(torch.float16)
# model = model.to("cuda")
model.eval()
step = checkpoint['step']
print(checkpoint['step'])

/home/steve/miniconda3/envs/gym/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


4000


In [3]:
audio, text = sampler()
print(audio[0], text[0])
encoded = audio[0]
text = text[0]
# id = "./external_datasets/libriheavy-encodec/9774/extermination_american_bison_1503_librivox_64kb_mp3/exterminationamericanbison_12_hornaday_64kb_99"
# with open(id + ".txt", 'r') as file:
#     text = file.read()
#     text = tokenizer.encode(text)
# encoded = torch.load(id + ".pt")

tensor([[ 835,  475,  835,  ...,  835,  408,  408],
        [ 646, 1023,  544,  ...,  518,  544,  544],
        [ 326,  678,  989,  ...,  989,  786,  678],
        ...,
        [ 903,  763,  293,  ...,  881,  986,  939],
        [ 905,  845,  900,  ...,  900,  900,  900],
        [ 617,  484,  475,  ...,  948,  975,  975]]) tensor([ 713,  155,  658,   14, 1634,   51,   18,   11,  154,   15,  170,   14,
        2002,  600,   32,  272,   73,   65,   95, 2679, 2454,   43,   29, 7273,
         197,   12])


In [4]:
def decode(tokens):
    features = vocos.codes_to_features(tokens)
    bandwidth_id = torch.tensor([2])  # 6 kbps
    return vocos.decode(features, bandwidth_id=bandwidth_id)
display(Audio(data=decode(encoded).cpu(), rate=24000))
print(encoded.shape)

torch.Size([8, 456])


In [5]:
condition_text = text
condition_audio = encoded[:,:75*3]
audio = encoded[:,75*3:]
predicted = [audio[0]]
print(audio[0])
for i in range(1, 8):
    p = model(
        condition_text = [condition_text], 
        condition_audio = [condition_audio],
        audio = [torch.stack(predicted)],
        codec = [i]
    )

    # Top-3
    p = p[0]
    v, _ = torch.topk(p.unsqueeze(0), 3)
    v = v.squeeze(0)
    p[p < v[:, [-1]]] = -float('Inf')

    # Sample
    p = torch.nn.functional.softmax(p, dim=-1)
    p = torch.multinomial(p, num_samples=1)
    p = p.squeeze(-1)

    print("inference")
    print(p)
    print(audio[i])
    predicted.append(p.cpu())
predicted = torch.stack(predicted)
predicted = torch.cat([condition_audio, predicted], dim = 1)
display(Audio(data=decode(predicted).cpu(), rate=24000))

tensor([ 372,  694,  372,  428,  428,  393,   30,   30,  976,  724,  339,  463,
         463,  463,  463,  463,  463,  463,  463,  463,  463,  463,  339,  339,
         121,   62,   62,   62,  408,  408,   62,  408,   62,  408,  408,  408,
         408,  408,  408,  432,  276,  151,   59,   59,  650,  582,  602,  780,
         876, 1017,  876,  430,  131,  325,  976,  131,  233,   59, 1008,  185,
         428,  155,   30,   30,  699,  871,  291,  582,  582,  428,   59,  192,
         323,  724,  976,  753, 1019, 1017,  408,  537,  677,  491,  958,   23,
        1019, 1017,  738,   62,  408,  408,  395,  537,  879,  379,  428,  233,
         372,  402,  694, 1022,  433,  228,  463,  463,  463,  635,  393,  724,
         228,  976,  276,   59,   59,  151,   23,   23,  999,  876,  738,  408,
          25,   38,  488,  537,  395,  779,  876,  293,  415,   59,   23,  276,
        1017,  834,  151,  871,  192,  753,  432,  699,  325,  871,  565,  192,
         602,  777,  430,  491,  783,   

In [6]:
# plt.imshow(model.positional_embedding.weight.cpu().detach().numpy(), cmap='hot', interpolation='nearest')
# plt.show()

In [7]:
# model_t = SupervoceNARModel()
# for i in range(1, 10):
#     checkpoint = torch.load("./output/valle-05." + str(step - i * 1000) + ".pt")
#     model_t.load_state_dict(checkpoint['model'])
#     model_t = model_t.to(torch.float16)
#     model_t = model_t.to("cuda")
#     predicted = [audio[0]]
#     for i in range(1, 8):
#         p = model_t(
#             condition_text = condition_text.to("cuda"), 
#             condition_audio = condition_audio.to("cuda"),
#             audio = audio.to("cuda"),
#             codec = i
#         )
#         p = torch.nn.functional.softmax(p, dim=-1)
#         p = torch.argmax(p, dim=-1)
#         predicted.append(p.cpu())
#     predicted = torch.stack(predicted)
#     predicted = torch.cat([condition_audio, predicted], dim = 1)
#     display(Audio(data=decode(predicted).cpu(), rate=24000))